# Intro to AI 4 - CW3 Artifical Neural Networks (ANN)

In [14]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

DATA_PATH = r"./interest-rates/index.csv"
df = pd.read_csv(DATA_PATH)

df["date"] = pd.to_datetime(df[["Year", "Month", "Day"]])
df = df.sort_values("date")

print(df.shape)
display(df.head())

if (torch.cuda.is_available()):
    print("CUDA available. Using device: ", torch.cuda.get_device_name())
else:
    print("CUDA is not available. Using device:: ", torch.device("cpu"))


(904, 11)


,Year,Month,Day,Federal Funds Target Rate,Federal Funds Upper Target,Federal Funds Lower Target,Effective Federal Funds Rate,Real GDP (Percent Change),Unemployment Rate,Inflation Rate,date
0,1954,7,1,NaN,NaN,NaN,0.80,4.6,5.8,NaN,1954-07-01
1,1954,8,1,NaN,NaN,NaN,1.22,NaN,6.0,NaN,1954-08-01
2,1954,9,1,NaN,NaN,NaN,1.06,NaN,6.1,NaN,1954-09-01
3,1954,10,1,NaN,NaN,NaN,0.85,8.0,5.7,NaN,1954-10-01
4,1954,11,1,NaN,NaN,NaN,0.83,NaN,5.3,NaN,1954-11-01


CUDA available. Using device:  NVIDIA GeForce RTX 3070


In [3]:
def prepare_quarterly_data(df_in: pd.DataFrame, add_lags: bool = False, max_lag: int = 2):
    target_col = "Real GDP (Percent Change)"
    base_feature_cols = [
        "Effective Federal Funds Rate",
        "Unemployment Rate",
        "Inflation Rate",
    ]

    # using a copy of the data
    temp = df_in.copy()
    temp = temp.set_index("date").sort_index()

    # FF between GDP and keep only relevant
    temp[target_col] = temp[target_col].ffill()
    cols_to_keep = [target_col] + base_feature_cols
    temp = temp[cols_to_keep]

    # Interpolate features in time, then back-fill any remaining NAs
    temp[base_feature_cols] = temp[base_feature_cols].interpolate(method="time")
    temp[base_feature_cols] = temp[base_feature_cols].bfill()

    df_q = temp.resample("QS", label="left").mean()

    # create feature lags on this quarterly index.
    feature_cols = base_feature_cols.copy()

    if add_lags:
        for col in base_feature_cols:
            for lag in range(1, max_lag + 1):
                lag_name = f"{col}_lag{lag}q"
                df_q[lag_name] = df_q[col].shift(lag)
                feature_cols.append(lag_name)

        df_q = df_q.dropna(subset=feature_cols + [target_col]) # Drop rows with inital NAs from the lags

    return df_q, feature_cols


In [4]:
# quarterly data with lags
pipeline_input_down_lag, feature_cols_down_lag = prepare_quarterly_data(
    df, add_lags=True, max_lag=2
)

print("Downsampled quarterly + feature lags shape:", pipeline_input_down_lag.shape)

display(pipeline_input_down_lag.head())

Downsampled quarterly + feature lags shape: (249, 10)


,Real GDP (Percent Change),Effective Federal Funds Rate,Unemployment Rate,Inflation Rate,Effective Federal Funds Rate_lag1q,Effective Federal Funds Rate_lag2q,Unemployment Rate_lag1q,Unemployment Rate_lag2q,Inflation Rate_lag1q,Inflation Rate_lag2q
date,,,,,,,,,,
1955-01-01,11.9,1.343333,4.733333,3.2,0.986667,1.026667,5.333333,5.966667,3.2,3.2
1955-04-01,6.7,1.500000,4.400000,3.2,1.343333,0.986667,4.733333,5.333333,3.2,3.2
1955-07-01,5.5,1.940000,4.100000,3.2,1.500000,1.343333,4.400000,4.733333,3.2,3.2
1955-10-01,2.4,2.356667,4.233333,3.2,1.940000,1.500000,4.100000,4.400000,3.2,3.2
1956-01-01,-1.5,2.483333,4.033333,3.2,2.356667,1.940000,4.233333,4.100000,3.2,3.2


In [5]:

data = pipeline_input_down_lag.sort_index()
data = data.reset_index(drop=True)

train_ratio = 0.70
val_ratio   = 0.15

N = len(data)

train_end = int(N * train_ratio)
val_end   = int(N * (train_ratio + val_ratio))

#Split!
train_df = data.iloc[:train_end]
val_df   = data.iloc[train_end:val_end]
test_df  = data.iloc[val_end:]

X_train = train_df[feature_cols_down_lag].values.astype(np.float32)
y_train = train_df["Real GDP (Percent Change)"].values.astype(np.float32)

X_val   = val_df[feature_cols_down_lag].values.astype(np.float32)
y_val   = val_df["Real GDP (Percent Change)"].values.astype(np.float32)

X_test  = test_df[feature_cols_down_lag].values.astype(np.float32)
y_test  = test_df["Real GDP (Percent Change)"].values.astype(np.float32)



### Standardisation

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)

# aplly transformation to all other feature sets
X_train_scaled = scaler.transform(X_train).astype(np.float32)
X_val_scaled   = scaler.transform(X_val).astype(np.float32)
X_test_scaled  = scaler.transform(X_test).astype(np.float32)

# Pytorch dataset class
class ff_dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = ff_dataset(X_train_scaled, y_train)
val_dataset   = ff_dataset(X_val_scaled, y_val)
test_dataset  = ff_dataset(X_test_scaled, y_test)

### Batches

In [7]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)